In [1]:
import numpy as np
from itertools import product
from collections import Counter
from datasets import load_dataset
from sklearn.metrics import accuracy_score, confusion_matrix

In [2]:
def metric_calculation(pred, gt):    
    acc=accuracy_score(gt, pred)    
    confusion=confusion_matrix(gt, pred)
    fpr=confusion[0,1]/len(gt) ## predict to be 1; actual 0
    fnr=confusion[1,0]/len(gt) ## predict to be 0; actual 1
    return acc, fpr, fnr

In [3]:
def post_processing(pred):
    new_pred=[]
    for i in pred:
        i=i.lower()
        if 'response' in i:
            try: new_pred.append(i.split('response')[1].split()[1].replace('</s>', ''))
            except: new_pred.append(2)
        elif 'output' in i:
            try: new_pred.append(i.split('output')[1].split()[1].replace('</s>', ''))
            except: new_pred.append(2)
        else:
            try: new_pred.append(i.split()[0].replace('</s>', ''))
            except:new_pred.append(2)
    new_pred = np.array([int(float(i)) if i in ['0', '0.0', '1', '1.0'] else 2 for i in new_pred])
    return new_pred

In [4]:
ds = load_dataset("beanham/spatial_join_dataset")
test=ds['test']
gt=np.array(test['label'])

In [16]:
base=np.load('base/4o_mini/4o_mini_few_shot_with_heur_value_angle.npy')
ec=np.load('ec/4o_mini/4o_mini_degree_1_few_shot_with_heur_value_angle_sc.npy')
base=post_processing(base)
ec=post_processing(ec)
print(f'Heuristics:                     ', metric_calculation(np.array(test['min_angle'])<=1, gt))
print(f'Base:                           ', metric_calculation(base, gt))
print(f'External Correction (degree=1): ', metric_calculation(ec, gt))

Heuristics:                      (0.8572825024437928, 0.03062886933854676, 0.11208862821766048)
Base:                            (0.9286412512218963, 0.06972955360052134, 0.0016291951775822744)
External Correction (degree=1):  (0.9429781687846204, 0.05311176278918214, 0.0039100684261974585)
